# fairness_toolkit Tutorial
## Assessing Intersectional Fairness in a Health Classification Model

This notebook demonstrates how to use the fairness_toolkit Python package to:

1. Load a medical dataset

2. Train a simple classification model

3. Create intersectional groups (e.g., sex × age group)

4. Compute Differential Fairness (ε)

5. Visualise fairness across subgroups

The purpose of this example is to show a typical health data science workflow where fairness matters — for example, predicting heart disease risk.

## 1. Install and import the toolkit

In [ ]:
from fairness_toolkit.groups import create_intersectional_groups
from fairness_toolkit.differential_fairness import differential_fairness
from fairness_toolkit.plots import plot_group_fairness
import pandas as pd
import numpy as np

## Load Data

In this example, we will use the Heart Disease dataset. Each row represents a patient. The target variable (target) indicates presence of heart disease (0 = no, 1 = yes).

In [ ]:
df = pd.read_csv('data/heart.csv')
df.head()

## Prepare the dataset

create age groups to use later as part of the intersectional fairness analysis

In [ ]:
df["age_group"] = pd.cut(
    df["age"],
    bins=[0, 40, 60, 120],
    labels=["young", "middle", "older"]
)

df[["age", "age_group", "sex"]].head()


Protected attributes:

protected = ["sex", "age_group"]


Train a simple classification model

Create intersectional groups from protected characteristics

Compute differential fairness